# Lecture: Monodromy

In [1]:
using HomotopyContinuation

<hr style="border:1px solid gray"> </hr>
For more information about monodromy and computing  monodromy groups using numerical algebraic geometry see 

[[*Numerical computation of Galois groups*](https://arxiv.org/pdf/1605.07806.pdf)] Hauenstein, Rodriguez, and Sottile. Foundations of Computational Mathematics, 18 (2018)

For more information about solve polynomial systems using monodromy see

[[*Solving polynomial systems via homotopy continuation and monodromy*](https://arxiv.org/pdf/1609.08722.pdf)]  Duff,  Hill,  Jensen,  Lee,  Leykin, and  Sommars.
IMA Journal of Num. Anal. (2018)

<hr style="border:1px solid gray"> </hr>

<img src="coveringspace.png" width="500" style="float:right">

## Branched Covers

As in Sascha's talk, we consider a family of $0$-dimensional polynomial systems parametrized by some variety $Q$ (let's just take $Q=\mathbb{C}^k$)
$$Z = \{(x,p) \in \mathbb{C}^n \times Q \mid F(x;p) =0\} \subset \mathbb{C}^n \times Q$$
$$\,\,\,\, \downarrow \pi $$
$$Q$$

Over *most* parameters, there are $d$ solutions.

The map $\pi$ is called a **branched cover** 

The **discriminant** (or branch locus) $D \subsetneq Q$ consists of the parameter values where the fibres don't have the expected count.

On $U = Q \backslash D$ (the regular values), the map $\pi:Z|_{\pi^{-1}(U)} \to U$ is a $d$-to-$1$ **covering space**.







### Example

$$F(x;a,b) = x^2+ax+b$$


In [2]:
@var x a b
F = System([x^2 + a * x + b], parameters = [a, b])

System of length 1
 1 variables: x
 2 parameters: a, b

 b + a*x + x^2

<p style="border:3px; border-style:solid; padding: 0.5em; text-align:center">
Monodromy:
As we move in $U$, the fibres move.
When we come back to where we started, the points in the fibre may permute.
</p>

In other words, we take loops $t \xrightarrow{\gamma} U$ with $\gamma(0)=\gamma(1)$ in the parameters space, and follow the fibres using homotopy continuation.

Let's move from $$(x^2-1) \to (-ix^2-1) \to (-x^2-1) \to (ix^2-1) \to (x^2-1)$$

<img src="monodromy1b.png" width="500">

In [3]:
S₀=[[-1],[1]]
p₀=[0,-1]

2-element Array{Int64,1}:
  0
 -1

In [4]:
p₁ = [0, im]
p₂ = [0,1]
p₃ = [0,-im]

2-element Array{Complex{Int64},1}:
 0 + 0im
 0 - 1im

In [5]:
#p0->p1
S₁ = solve(F,S₀; start_parameters=p₀,target_parameters=p₁)
#p1->p2
S₂ = solve(F,S₁; start_parameters=p₁,target_parameters=p₂)
#p2->p3
S₃ = solve(F,S₂; start_parameters=p₂,target_parameters=p₃)
#p3->p0 again
S₀2 = solve(F,S₃; start_parameters=p₃,target_parameters=p₀)

Tracking 2 paths... 100%|███████████████████████████████| Time: 0:00:05
  # paths tracked:                  2
  # non-singular solutions (real):  2 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         2 (0)


Result with 2 solutions
• 2 paths tracked
• 2 non-singular solutions (2 real)
• random_seed: 0xca7526e3


In [6]:
show(stdout,"text/plain",hcat(S₀,solutions(S₁),solutions(S₂),solutions(S₃),solutions(S₀2)))

2×5 Array{Array{Complex{Float64},1},2}:
 [-1.0+0.0im]  [-0.707107+0.707107im]  [0.0+1.0im]  [0.707107+0.707107im]   [1.0+0.0im]
 [1.0+0.0im]   [0.707107-0.707107im]   [0.0-1.0im]  [-0.707107-0.707107im]  [-1.0+0.0im]

Note that the solutions have permuted.
<img src="monodromy1b.png" width="500">

In [7]:
#This function automates the process above, given a parametrized polynomial system F, start solutions 
# Sols, and a loop γ which is just a sequence of parameter values starting and ending with the parameter
# value for Sols

function monodromy_loop(F,Sols,γ)
    S=Sols;
    P=γ[1]
    for NewP in γ[2:end]
        newS=solve(F,S; start_parameters=P, target_parameters=NewP)
        S=newS
        P=NewP
    end
    return S
end

monodromy_loop (generic function with 1 method)

In [8]:
#Now let's take the loop again and see that we return to the solutions [[-1],[1]] in order.
M=monodromy_loop(F,solutions(S₀2),[p₀,p₁,p₂,p₃,p₀]);
solutions(M)

2-element Array{Array{Complex{Float64},1},1}:
 [-1.0 + 0.0im]
 [1.0 + 0.0im]

### Solving systems using the idea  of monodromy

What if we only knew about the solution $s_1=1$ of $x^2-1=0$?

Then performing the monodromy loop *discovers* the solution $s_2=-1$

In [9]:
solutions(monodromy_loop(F,[1],[p₀,p₁,p₂,p₃,p₀]))

1-element Array{Array{Complex{Float64},1},1}:
 [-1.0 + 0.0im]

What if we didn't know any solutions to a polynomial $f(x) = \sum_{i=0}^{20}c_ix^i$?

In [10]:
@var c[0:20]
f = sum([c[i]*x^(i-1) for i in 1:21])
F = System([f],parameters=c)

System of length 1
 1 variables: x
 21 parameters: c₀, c₁, c₂, c₃, c₄, c₅, c₆, c₇, c₈, c₉, c₁₀, c₁₁, c₁₂, c₁₃, c₁₄, c₁₅, c₁₆, c₁₇, c₁₈, c₁₉, c₂₀

 c₀ + x*c₁ + x^2*c₂ + x^3*c₃ + x^4*c₄ + x^5*c₅ + x^6*c₆ + x^7*c₇ + x^8*c₈ + x^9*c₉ + x^10*c₁₀ + x^11*c₁₁ + x^12*c₁₂ + x^13*c₁₃ + x^14*c₁₄ + x^15*c₁₅ + x^16*c₁₆ + x^17*c₁₇ + x^18*c₁₈ + x^19*c₁₉ + x^20*c₂₀

We would need to find a single start solution

In [11]:
#So we can force the start solution to be 0.923+0.2im
S=[0.923+0.2*im]
C = randn(ComplexF64,21)
evaluation = sum([C[i]*S[1]^(i-1) for i in 1:21])
C[1]-=evaluation ##Synthetically alters the tuple C so that S[1] is a solution to f_C(x)
evaluation = sum([C[i]*S[1]^(i-1) for i in 1:21])

2.498001805406602e-16 + 7.632783294297951e-17im

Now we have 1 solution to some degree 19 polynomial 

In [12]:
SolutionsFound=[S]
loop_num=0
while length(SolutionsFound)<20
    loop_num+=1
    newSolution=solutions(monodromy_loop(F,S,[C,5*randn(ComplexF64,21),3*randn(ComplexF64,21),C]))
    if findfirst(x->x[1]≈(newSolution[1][1]),SolutionsFound)==nothing
        push!(SolutionsFound,newSolution[1])
        println("We found a new solution (Total:",length(SolutionsFound),")")
        flush(stdout)
    end
end
println("Finished after ",loop_num, " loops")
    

We found a new solution (Total:2)
We found a new solution (Total:3)
We found a new solution (Total:4)
We found a new solution (Total:5)
We found a new solution (Total:6)
We found a new solution (Total:7)
We found a new solution (Total:8)
We found a new solution (Total:9)
We found a new solution (Total:10)
We found a new solution (Total:11)
We found a new solution (Total:12)
We found a new solution (Total:13)
We found a new solution (Total:14)
We found a new solution (Total:15)
We found a new solution (Total:16)
We found a new solution (Total:17)
We found a new solution (Total:18)
We found a new solution (Total:19)
We found a new solution (Total:20)
Finished after 809 loops


<img src="dumbmonodromy.png" width="500">

All of this is implemented in HomotopyContinuation.jl in a smarter way following ideas in Duff et.al.

In [13]:
MonSolve=monodromy_solve(F,S,C; show_progress=true)

MonodromyResult
• return_code → :heuristic_stop
• 20 solutions
• 200 tracked loops
• random_seed → 0x80b83d82

<img src="smartermonodromy.png" width="500">

Let's check that we got the same answers

In [14]:
sort(solutions(MonSolve);lt=(x,y)->abs(real(x[1]))<abs(real(y[1])))

20-element Array{Array{Complex{Float64},1},1}:
 [0.10863360548844195 - 1.0513573890447196im]
 [0.20507760791791202 + 1.0132987935057682im]
 [-0.22190547848199993 + 0.9376465369248143im]
 [-0.2677316717217432 - 1.0057525196678987im]
 [0.3136815570806021 - 0.9569450840618844im]
 [-0.38822307387360017 + 1.7400833722239837im]
 [0.5532859744850019 - 1.8646316802332148im]
 [0.5561754675319385 + 0.9101362085413984im]
 [-0.6470889941660677 + 0.9412054338640071im]
 [-0.6632688132567715 - 0.7815020948302681im]
 [0.8056374150853376 + 0.5826823618384008im]
 [0.8539164541792945 - 0.6205937658522558im]
 [-0.9009828615714951 + 0.41127684955768123im]
 [0.923 + 0.2im]
 [0.9333139114507538 - 0.28987891954481454im]
 [-0.9542174462126553 - 0.4350700866803588im]
 [1.0518575534854473 - 0.0938361933075682im]
 [-1.0567389507697498 + 0.036451169678619166im]
 [-1.3072949818535493 + 0.9431837286928044im]
 [-1.3784510820108007 - 0.5725498558972494im]

In [15]:
sort(SolutionsFound;lt=(x,y)->abs(real(x[1]))<abs(real(y[1])))

20-element Array{Array{Complex{Float64},1},1}:
 [0.10863360548844195 - 1.0513573890447196im]
 [0.20507760791791205 + 1.0132987935057682im]
 [-0.22190547848199993 + 0.9376465369248143im]
 [-0.2677316717217432 - 1.0057525196678987im]
 [0.3136815570806021 - 0.9569450840618844im]
 [-0.3882230738736 + 1.740083372223984im]
 [0.5532859744850019 - 1.8646316802332146im]
 [0.5561754675319385 + 0.9101362085413984im]
 [-0.6470889941660676 + 0.9412054338640071im]
 [-0.6632688132567715 - 0.7815020948302681im]
 [0.8056374150853376 + 0.5826823618384008im]
 [0.8539164541792945 - 0.6205937658522558im]
 [-0.9009828615714951 + 0.41127684955768123im]
 [0.923 + 0.2im]
 [0.9333139114507538 - 0.28987891954481454im]
 [-0.9542174462126553 - 0.4350700866803588im]
 [1.0518575534854473 - 0.0938361933075682im]
 [-1.0567389507697498 + 0.03645116967861917im]
 [-1.3072949818535493 + 0.9431837286928044im]
 [-1.3784510820108007 - 0.5725498558972494im]

<bdi style="border:3px; border-style:solid; padding: 0.2em;">
Warning! This may not find all the points you're looking for!
</bdi>

Situation 1: you were unlucky and the loops you took never produced all of the solutions

Situation 2: no matter which loops you took, you'd never produce all the solutions

Example for situation 2: $Z=Z_1 \cup Z_2$ and your start point is $(x,p)$ where $x \in Z_1$, then monodromy will never produce a point on $Z_2$.

In [16]:
@var x,y
@var a,b,c
TwoConics = System([(x^2+y^2-1)*((x-0.5)^2+y^2-1),a*x+b*y+c],parameters=[a,b,c])
M=monodromy_solve(TwoConics,[[1,0]],[0,1,0])

MonodromyResult
• return_code → :heuristic_stop
• 2 solutions
• 18 tracked loops
• random_seed → 0x56994b8e

## Monodromy Group

The collection of all permutations possible (from loops based at $p \in U$) forms a group called the **monodromy group** of $\pi$.

Exercise: show this is a group and that it doesn't depend on $p \in U$.

The following function writes the permutation $g_\gamma$ coming from a loop $\gamma$ as a permutation in single line notation

In [17]:
#This function will produce the permutation on the fibre BaseSols (over BaseParam) induced by the loop γ
#  which is just a sequence of parameter values starting and ending with BaseParam
#  If no such loop is given, it will randomly make one

function monodromy_element(F,BaseSols,BaseParam;γ=nothing)
    #If no loop is given, randomly make one
    if γ==nothing
        γ = [BaseParam] #starting at BaseParam
        for i in 1:3
            push!(γ,2*randn(ComplexF64,length(F.parameters))) #with 3 other parameter values
        end
        push!(γ,BaseParam) #and ending again at BaseParam
    end
    M = solutions(monodromy_loop(F,BaseSols,γ)) #Take the solutions after the loop
    S = (BaseSols) #And those before
    gᵧ= [findfirst(x->x≈S[i],M) for i in 1:length(M)] #and see which solutions were permuted
    return gᵧ
end

monodromy_element (generic function with 1 method)

In [18]:
@var x a b
F = System([x^2 + a * x + b], parameters = [a, b])
#Here's the monodromy element from before written as a permutation
monodromy_element(F,[[-1],[1]],p₀;γ=[p₀,p₁,p₂,p₃,p₀])

2-element Array{Int64,1}:
 2
 1

Let's compute 1000 random monodromy permutations!

In [19]:
#here are 1000 random monodromy elements based at p_0
random_elements=[monodromy_element(F,[[-1],[1]],Array{ComplexF64,1}(p₀)) for i in 1:1000];

In [20]:
#this is a quick function to tally them
function tally(S)
    D=Dict{Any,Int}()
    for s in S
        D[s]=get(D,s,0)+1
    end
    return D
end

tally (generic function with 1 method)

In [21]:
#here's the tally
tally(random_elements)

Dict{Any,Int64} with 2 entries:
  [2, 1] => 337
  [1, 2] => 663

Exercise: monodromy_solve can find all solutions if and only if $G_\pi$ is transitive

Exercise: $G_\pi$ is transitive if and only if $Z$ has a unique irreducible component of top dimension

# A nontrivial example: 27 lines on a cubic

**Cayley and Salmon**: There are exactly $27$ lines on every smooth cubic surface in $\mathbb{P}^3$.

Let's solve for these $27$ lines using monodromy, then compute the monodromy group!

*Source: Greg Egan* 

<img src="clebsch_diagonal_cubic-1-1.gif" width="500">
(The Clebsch cubic, a cubic surface with all 27 lines real; it is also special, as it has Eckhardt points)

### Writing the system 

Cubic = $\sum_{i+j+k\leq 3} c_{i,j,k}x^iy^jz^k$

Lines = $t \xrightarrow{\ell_{k}} (t,k_1+k_3t,k_2+k_4t)$

Polynomial system: Cubic($\ell_k$)$=0$ (variables are $k_1,\ldots,k_4$ and parameters are $c_{i,j,k}$)

In [22]:
# lines are parametrized as t->(t,k1+k3t,k2+k4t)
@var k[1:4]
# the coefficient of x^iy^jz^k in a general cubic will be c[i,j,k]
@var c[0:3,0:3,0:3]
# we need these variables too
@var t
@var x,y,z

#here's a cubic
cubic=sum([c[i+1,j+1,k+1]x^i*y^j*z^k for i in 0:3 for j in 0:3-i for k in 0:3-i-j])
#here's our line
line = [t,k[1]+k[3]*t,k[2]+k[4]*t]
#E is cubic(line(t))
E=evaluate(cubic,[x,y,z]=>line)
#and we construct the system where the coefficients of t^i for t=0..3 are zero (i.e. E=0)
F = System(exponents_coefficients(E,[t])[2],parameters=vec(c))

System of length 4
 4 variables: k₁, k₂, k₃, k₄
 64 parameters: c₀₋₀₋₀, c₁₋₀₋₀, c₂₋₀₋₀, c₃₋₀₋₀, c₀₋₁₋₀, c₁₋₁₋₀, c₂₋₁₋₀, c₃₋₁₋₀, c₀₋₂₋₀, c₁₋₂₋₀, c₂₋₂₋₀, c₃₋₂₋₀, c₀₋₃₋₀, c₁₋₃₋₀, c₂₋₃₋₀, c₃₋₃₋₀, c₀₋₀₋₁, c₁₋₀₋₁, c₂₋₀₋₁, c₃₋₀₋₁, c₀₋₁₋₁, c₁₋₁₋₁, c₂₋₁₋₁, c₃₋₁₋₁, c₀₋₂₋₁, c₁₋₂₋₁, c₂₋₂₋₁, c₃₋₂₋₁, c₀₋₃₋₁, c₁₋₃₋₁, c₂₋₃₋₁, c₃₋₃₋₁, c₀₋₀₋₂, c₁₋₀₋₂, c₂₋₀₋₂, c₃₋₀₋₂, c₀₋₁₋₂, c₁₋₁₋₂, c₂₋₁₋₂, c₃₋₁₋₂, c₀₋₂₋₂, c₁₋₂₋₂, c₂₋₂₋₂, c₃₋₂₋₂, c₀₋₃₋₂, c₁₋₃₋₂, c₂₋₃₋₂, c₃₋₃₋₂, c₀₋₀₋₃, c₁₋₀₋₃, c₂₋₀₋₃, c₃₋₀₋₃, c₀₋₁₋₃, c₁₋₁₋₃, c₂₋₁₋₃, c₃₋₁₋₃, c₀₋₂₋₃, c₁₋₂₋₃, c₂₋₂₋₃, c₃₋₂₋₃, c₀₋₃₋₃, c₁₋₃₋₃, c₂₋₃₋₃, c₃₋₃₋₃

 c₃₋₀₋₀ + k₃*c₂₋₁₋₀ + k₃^2*c₁₋₂₋₀ + k₃^3*c₀₋₃₋₀ + k₄*c₂₋₀₋₁ + k₄^2*c₁₋₀₋₂ + k₄^3*c₀₋₀₋₃ + k₄*k₃*c₁₋₁₋₁ + k₄*k₃^2*c₀₋₂₋₁ + k₄^2*k₃*c₀₋₁₋₂
 c₂₋₀₋₀ + k₁*c₂₋₁₋₀ + k₂*c₂₋₀₋₁ + k₃*c₁₋₁₋₀ + k₃^2*c₀₋₂₋₀ + k₄*c₁₋₀₋₁ + k₄^2*c₀₋₀₋₂ + 2*k₃*k₁*c₁₋₂₋₀ + k₃*k₂*c₁₋₁₋₁ + 3*k₃^2*k₁*c₀₋₃₋₀ + k₃^2*k₂*c₀₋₂₋₁ + k₄*k₁*c₁₋₁₋₁ + 2*k₄*k₂*c₁₋₀₋₂ + k₄*k₃*c₀₋₁₋₁ + k₄^2*k₁*c₀₋₁₋₂ + 3*k₄^2*k₂*c₀₋₀₋₃ + 2*k₄*k₃*k₁*c₀₋₂₋₁ + 2*k₄*k₃*k₂*c₀₋₁₋₂
 c₁₋₀₋₀ + k₁

In [23]:
@time MS=monodromy_solve(F)

  7.150276 seconds (14.08 M allocations: 697.161 MiB, 3.93% gc time)


MonodromyResult
• return_code → :heuristic_stop
• 27 solutions
• 216 tracked loops
• random_seed → 0x09179a14

Monodromy quickly solved the system!

In [24]:
#Output of monodromy_solve knows a fibre pi^(-1)(P) = S
S=solutions(MS);
P=parameters(MS);

In [25]:
@time random_elements=[monodromy_element(F,S,P) for i in 1:50];

  2.836459 seconds (6.27 M allocations: 307.807 MiB, 3.09% gc time)


In [26]:
#Here we filter out anything funny that happened numerically (note we weren't particularly
#  careful with our code earlier)
filter!(x->!(nothing in x) && sort(x) == collect(1:27),random_elements);
length(random_elements)

48

In [27]:
tally(random_elements)

Dict{Any,Int64} with 48 entries:
  [20, 24, 6, 3, 9, 4, 13, 11, 2, 10  …  27, 23, 1, 19, 25, 21, 8, 22, 15,… => 1
  [2, 11, 3, 19, 1, 25, 13, 27, 16, 10  …  15, 22, 14, 21, 23, 6, 5, 4, 24… => 1
  [27, 17, 2, 8, 11, 1, 20, 9, 15, 12  …  24, 7, 10, 6, 5, 3, 14, 4, 26, 2… => 1
  [18, 8, 21, 7, 15, 23, 25, 5, 26, 10  …  14, 4, 16, 3, 19, 13, 12, 6, 27… => 1
  [11, 2, 10, 17, 1, 9, 8, 4, 22, 3  …  6, 16, 20, 21, 18, 15, 24, 27, 5, … => 1
  [6, 24, 1, 20, 13, 4, 11, 22, 7, 23  …  19, 10, 3, 17, 2, 27, 25, 9, 14,… => 1
  [19, 13, 22, 11, 21, 17, 9, 27, 2, 25  …  8, 24, 4, 10, 20, 18, 7, 12, 6… => 1
  [23, 14, 15, 16, 24, 18, 10, 26, 7, 3  …  20, 27, 5, 17, 8, 9, 6, 12, 1,… => 1
  [3, 11, 20, 2, 22, 7, 17, 19, 25, 13  …  21, 10, 4, 16, 8, 18, 6, 26, 9,… => 1
  [20, 11, 4, 9, 2, 8, 27, 10, 18, 26  …  16, 13, 17, 15, 21, 25, 7, 1, 19… => 1
  [12, 19, 11, 13, 26, 24, 3, 23, 15, 16  …  5, 4, 14, 6, 1, 20, 10, 22, 2… => 1
  [21, 12, 22, 19, 9, 4, 11, 27, 25, 15  …  1, 26, 13, 2, 3, 20, 6, 17, 24… 

#### To compute and describe this monodromy group, we'll need a little help from our favorite group theory software

In [28]:
using GAP;
function describe_group(L)
    L=unique(L)
    Sym=GAP.Globals.SymmetricGroup(length(L[1]))
    GAPL=[]
    for l in L
        push!(GAPL,GAP.Globals.PermList(GAP.julia_to_gap(l)))
    end
    GAPL=GAP.julia_to_gap(GAPL)
    G=GAP.Globals.Subgroup(Sym,GAPL)
    return (GAP.Globals.StructureDescription(G),G)
end

 ┌───────┐   GAP 4.11.0 of 29-Feb-2020
 │  GAP  │   https://www.gap-system.org
 └───────┘   Architecture: x86_64-pc-linux-gnu-julia64-kv7-v1.5
 Configuration:  gmp 6.1.2, Julia GC, Julia 1.5.2, readline
 Loading the library and packages ...
 Packages:   AClib 1.3.2, Alnuth 3.1.2, AtlasRep 2.1.0, AutoDoc 2019.09.04, 
             AutPGrp 1.10.2, CRISP 1.4.5, Cryst 4.1.23, CrystCat 1.1.9, 
             CTblLib 1.2.2, FactInt 1.6.3, FGA 1.4.0, Forms 1.2.5, 
             GAPDoc 1.6.3, genss 1.6.6, IO 4.7.0, IRREDSOL 1.4, LAGUNA 3.9.3, 
             orb 4.8.3, Polenta 1.3.9, Polycyclic 2.15.1, PrimGrp 3.4.0, 
             RadiRoot 2.8, recog 1.3.2, ResClasses 4.7.2, SmallGrp 1.4.1, 
             Sophus 1.24, SpinSym 1.5.2, TomLib 1.2.9, TransGrp 2.0.5, 
             utils 0.69
 Try '??help' for help. See also '?copyright', '?cite' and '?authors'


describe_group (generic function with 1 method)

In [29]:
Gal=describe_group(random_elements)
println(Gal[1])
GAP.Globals.Order(Gal[2])

GAP: "O(5,3) : C2"


51840

##### This is the Weyl group of E6

It is exactly the group of permutations which are possible given how the $27$ lines must intersect.

General philosophy: the monodromy group is always as big as possible given the geometric obstructions (not actually a mathematical statement)

## Conclusion
- we can solve systems using monodromy provided we know the incidence variety is irreducible
 - How do we know when to stop? (stay tuned for witness sets talk)
- we can compute monodromy groups which reveal structure about our system
 - Find your favorite family of zero-dimensional varieties and compute their monodromy groups
   - Schubert Problems
   - Sparse polynomial systems
   - Computer vision